In [49]:
import json  
import pandas as pd
import Utilities
import copy as cp
import numpy as np
import matplotlib.pyplot as plt
import csv

In [ ]:
def convertAngles(theta):
    "convert angles from -180, 180 to 0, 360 and then to radians"
    if theta < 0:
        theta = 360 + theta
    return np.radians(theta)

In [50]:
## GENERAL ##
maxDist = 280
maxX = 1920
maxY = 1080
mapPath =  "/Users/Mattia/Desktop/EYE_TRACKER/MAPS/"
OUTPUT_PATH = "/Users/Mattia/Desktop/PAPER2/COARTICULATION/"

with open(OUTPUT_PATH + "subjNames.txt", "r") as f:
    subjNames = f.read().splitlines()
with open(OUTPUT_PATH + "problemList.txt", "r") as f:
    problemList = f.read().splitlines()
with open(OUTPUT_PATH + "graphNodeIdDict.json", "r") as f:
    graphNodeIdDict = json.load(f)
with open(OUTPUT_PATH + "mapIndex2ProblemV2.json", "r") as f:
    mapIndex2Problem = json.load(f)

In [67]:
angles = []
nextFixAngles = []
xpos = []
ypos = []
xposNext = []
yposNext = []
startNodes = []
endNodes = []
trialLs = []
labelFixLst = []
output = []
distance = []
distanceNext = []

#Random model 4
randomPositionX4 = []
randomPositionY4 = []
randomAngle4 = []
#Random model 5
randomPositionX5 = []
randomPositionY5 = []
randomAngle5 = []


RESTRICTED_CASE = True

RANDOM4 = True
RANDOM5 = True


#Load the graphs:
graphDict = {problem[:-5]: Utilities.map_load(mapPath + problem[:-5]) for problem in problemList}
R4Samples = pd.DataFrame()
R5Samples = pd.DataFrame()
N_SAMPLES = 1

for subjName in subjNames[:1]:
    print(subjName)
    df = pd.read_csv(OUTPUT_PATH + str(subjName) + "dfSampleVis.csv", low_memory=False)
    df = df.dropna(subset=["LABEL_FIX"])
    dfSubj = pd.read_csv(OUTPUT_PATH + str(subjName) + "dfTrajectoryVisResampledV2.csv", low_memory=False)

    if RESTRICTED_CASE:
        #restricted case condition
        condition = df["motiveNodes"] != "NOTHING"
        filtered_df = df[condition][["EYE_FIRST_CLOSEST_NODE", "motiveNodes"]].apply(lambda x: x["EYE_FIRST_CLOSEST_NODE"] == str(Utilities.parse_string(x["motiveNodes"])[-1]), axis = 1)
        result_df = df[condition][filtered_df]

        #Get the combination of TRIAL_INDEX and LABEL_FIX in result_df
        restrictedValues = list(result_df.groupby(["TRIAL_INDEX", "LABEL_FIX"]).groups.keys())
        #IMPORT
        dfSubj = pd.read_csv(OUTPUT_PATH + str(subjName) + "dfTrajectoryVisResampledV2.csv", low_memory=False)
        #in dfSubj drop all the rows whose TRIAL_INDEX and LABEL_FIX are not in restrictedValues
        dfSubj = dfSubj[dfSubj.apply(lambda x: (x["TRIAL_INDEX"], x["LABEL_FIX"]) in restrictedValues, axis = 1)]

    movementMask = df.BEFORE_MOVEMENT == False
    draggingMask = df.DRAGGING == True

    for TRIAL_INDEX in dfSubj.TRIAL_INDEX.unique().tolist():
        trialMask  = df.TRIAL_INDEX == TRIAL_INDEX 

        #Get the corresponding problem
        problem = df.loc[trialMask, "MAPNAME"].values[0]
        graph = graphDict[problem]

        for LABEL_FIX in dfSubj[dfSubj.TRIAL_INDEX == TRIAL_INDEX].LABEL_FIX.unique():
            labelMask = df.LABEL_FIX == LABEL_FIX
            try:
                #Get the last node of the motif
                endNode = Utilities.parse_string(df.loc[trialMask & labelMask, "almostActiveNode"].values[-1])[-1]
                startNode = Utilities.parse_string(df.loc[trialMask & labelMask, "almostActiveNode"].values[0])[-1]
            except:
                continue

            #Get the average eye position for current fixation
            avgx = df.loc[trialMask & labelMask, "avgx"].values[0]
            avgy = df.loc[trialMask & labelMask, "avgy"].values[0]

            #Find the angle of the average eye position w.r.t. the last node of the motif  TBC
            xEnd, yEnd = Utilities.mapGraph2Screen(problem, [(endNode[0], endNode[1])])
            #xEnd, yEnd = Utilities.mapGraph2Screen(problem, [(startNode[0], startNode[1])])

            #CURRENT FIXATION
            #Set the origin of the coordinate system in the last node of the motif
            avgxCentered = avgx - xEnd
            avgyCentered = avgy - yEnd
            currentFixAngle = np.arctan2(avgyCentered, avgxCentered)

            #Get the distance of the average eye position from the last node of the motif
            dist = np.sqrt(avgxCentered**2 + avgyCentered**2)
            #Check if the distance is greater than the maxDist (is it correct? TBC)
            if dist > maxDist:
                continue
            
            #NEXT FIXATION
            #Get the posizion of the next fixation
            try:
                nextFixX = df.loc[trialMask & movementMask & draggingMask & ( df.LABEL_FIX == LABEL_FIX + 1 ), "avgx"].values[0]
                nextFixY = df.loc[trialMask & movementMask & draggingMask & ( df.LABEL_FIX == LABEL_FIX + 1 ), "avgy"].values[0]
            except:
                continue

            #print("Start Node: ", startNode)
            #print("End Node: ", endNode)
            
            nextFixXCentered = nextFixX - xEnd
            nextFixYCentered = nextFixY - yEnd
            nextFixAngle = np.arctan2(nextFixYCentered, nextFixXCentered)
            
            #### RANDOM MODELS
            visitedNodes = df.loc[trialMask & labelMask, "activeNode"].values[0]
            visitedNodes = Utilities.parse_string(visitedNodes)
            #Get the nodes in the graph that have not been visited
            nodesTot = [tuple(node) for node in graphNodeIdDict[problem + ".json"]]
            notVisitedNodes = list(set(nodesTot) - set(visitedNodes))
            #Remaining rewards
            rewards = [tuple(node) for node in graphNodeIdDict[problem + ".json"] if graph.nodes[tuple(node)]["reward"] == 1 and tuple(node) != startNode and tuple(node) != endNode]
            remainingRewards = list(set(rewards) & set(notVisitedNodes))

            if len(remainingRewards) == 0:
                continue
            
            if RANDOM4:
                #RANDOM NODE POSITION NOT VISITED YET
                #Get the nodes in the graph that have  been visited 
                randomSample = []
                #Select a random node from the not visited nodes
                for i in range(N_SAMPLES):
                    randomNode = notVisitedNodes[np.random.randint(0, len(notVisitedNodes))]
                    #print("     Random Node: ", randomNode)
                    randomNode = Utilities.mapGraph2Screen(problem, [randomNode])
                    x4 = randomNode[0] - xEnd
                    y4 = randomNode[1] - yEnd
                    #Add random noise to the position
                    randomPositionX4.append(x4 + np.random.randint(-30, 30))
                    randomPositionY4.append(y4 + np.random.randint(-30, 30))
                    randomSample.append(np.degrees(np.arctan2(randomPositionY4[-1], randomPositionX4[-1])))    
                R4Samples = pd.concat([R4Samples, pd.DataFrame(randomSample)], axis = 1)

            if RANDOM5:
                #RANDOM REWARD POSITION NOT VISITED YET
                #Select a random node from the remaining rewards
                randomSample = []
                for i in range(N_SAMPLES):
                    randomNode = remainingRewards[np.random.randint(0, len(remainingRewards))]
                    #print("     Random Reward: ", randomNode)
                    randomNode = Utilities.mapGraph2Screen(problem, [randomNode])
                    x5 = randomNode[0] - xEnd
                    y5 = randomNode[1] - yEnd
                    #Add random noise to the position
                    randomPositionX5.append(x5 + np.random.randint(-30, 30))
                    randomPositionY5.append(y5 + np.random.randint(-30, 30))
                    randomSample.append(np.degrees(np.arctan2(randomPositionY5[-1], randomPositionX5[-1]))) 
                R5Samples = pd.concat([R5Samples, pd.DataFrame(randomSample)], axis = 1)

            #Convert the angles in degrees
            currentFixAngle = np.degrees(currentFixAngle)
            nextFixAngle = np.degrees(nextFixAngle)

            xpos.append(avgx)
            ypos.append(avgy)
            xposNext.append(nextFixX)
            yposNext.append(nextFixY)

            angles.append(currentFixAngle)
            nextFixAngles.append(nextFixAngle)
            startNodes.append(Utilities.mapGraph2Screen(problem, [startNode]))
            endNodes.append(Utilities.mapGraph2Screen(problem, [endNode]))
            labelFixLst.append(LABEL_FIX)
            trialLs.append(TRIAL_INDEX)
            distance.append(dist)

            start = df.loc[trialMask & labelMask, "t"].values[0]
            end = df.loc[trialMask & (df.LABEL_FIX == LABEL_FIX +1), "t"].values[-1]
            
            output.append(["python StandAloneVisualization.py PRIVATE " + str(TRIAL_INDEX) + " " + subjName + "" + " " + str(start) + " " + str(end)])

STUD4


In [130]:
#Save dataframes to csv
R4Samples.to_csv("./CCREG/R4Samples.csv", index = False)
R5Samples.to_csv("./CCREG/R5Samples.csv", index = False)
pd.Series(angles).to_csv("./CCREG/current_fixation_angle.csv", index = False)
pd.Series(nextFixAngles).to_csv("./CCREG/next_fixation_angle.csv", index = False)
#Save output to txt
with open("./CCREG/output.txt", "w") as f:
    for line in output:
        f.write(line[0] + "\n")

In [148]:
#Apply the conversion to the angles
currentFixationAngles = [convertAngles(angle) for angle in angles]
NextFixationAngles = [convertAngles(angle) for angle in nextFixAngles]
#Apply the convertAngles function to all the cells in the dataframe
R4Samples2 = R4Samples.applymap(convertAngles)
R5Samples2 = R5Samples.applymap(convertAngles)
indices = list(range(0, len(R4Samples2.columns)))

In [149]:
currentFixationAngles, NextFixationAngles, indices  = zip(*sorted(zip(currentFixationAngles, NextFixationAngles, indices)))

In [153]:
#Sort the columns of the dataframes
R4Samples2 = R4Samples2.iloc[:, list(indices)]
R5Samples2 = R5Samples2.iloc[:, list(indices)]

In [154]:
R4Samples2.to_csv("./CCREG/R4SamplesSorted.csv", index = False)
R5Samples2.to_csv("./CCREG/R5SamplesSorted.csv", index = False)
pd.Series(currentFixationAngles).to_csv("./CCREG/current_fixation_angleSorted.csv", index = False)
pd.Series(NextFixationAngles).to_csv("./CCREG/next_fixation_angleSorted.csv", index = False)